In [1]:
spark

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 6, Finished, Available, Finished)

In [2]:
from azureml.core import Workspace, Dataset, Datastore

subscription_id = '21ff0fc0-dd2c-450d-93b7-96eeb3699b22'
resource_group = 'project-group-29'
workspace_name = 'project-group-29'

workspace = Workspace(subscription_id, resource_group, workspace_name)

datastore = Datastore.get(workspace, "workspaceblobstore")
dataset = Dataset.Tabular.from_parquet_files(path=(datastore, 'states/states_sentiment'))
df = dataset.to_spark_dataframe()
df.head()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 7, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AUDVU2D92 to authenticate.
Interactive authentication successfully completed.
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '23137735-185a-436e-b332-97b921c8fadf'}
{'infer_column_types': 'False', 'activity': 'to_spark_dataframe', 'activityApp': 'TabularDataset', 'runId': '23137735-185a-436e-b332-97b921c8fadf', 'run_id': '23137735-185a-436e-b332-97b921c8fadf'}


Row(author='fireweedflowers', text='I live in Colorado these days but I was born and raised in Wyoming and was emphatically a socialist.', controversiality=0, created_utc=1701619261, distinguished=None, edited=None, gilded=0, id='kbtqz1h', parent_id='t3_187jmr5', score=1, subreddit='wyoming', subreddit_id='t5_2r53d', alaska=False, alabama=False, arkansas=False, american samoa=False, arizona=False, california=False, colorado=True, connecticut=False, district of columbia=False, delaware=False, florida=False, georgia=False, guam=False, hawaii=False, iowa=False, idaho=False, illinois=False, indiana=False, kansas=False, kentucky=False, louisiana=False, massachusetts=False, maryland=False, maine=False, michigan=False, minnesota=False, missouri=False, mississippi=False, montana=False, north carolina=False, north dakota=False, nebraska=False, new hampshire=False, new jersey=False, new mexico=False, nevada=False, new york=False, ohio=False, oklahoma=False, oregon=False, pennsylvania=False, puer

In [3]:
#df.printSchema()
df.show()

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 8, Finished, Available, Finished)

+--------------------+--------------------+----------------+-----------+-------------+------+------+-------+----------+-----+-------------------+------------+------+-------+--------+--------------+-------+----------+--------+-----------+--------------------+--------+-------+-------+-----+------+-----+-----+--------+-------+------+--------+---------+-------------+--------+-----+--------+---------+--------+-----------+-------+--------------+------------+--------+-------------+----------+----------+------+--------+-----+--------+------+------------+-----------+------------+--------------+------------+---------+-----+-----+--------+--------------+-------+----------+---------+-------------+-------+-------------+
|              author|                text|controversiality|created_utc|distinguished|edited|gilded|     id| parent_id|score|          subreddit|subreddit_id|alaska|alabama|arkansas|american samoa|arizona|california|colorado|connecticut|district of columbia|delaware|florida|georgia|

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression

state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", 
"Colorado", "Connecticut", "District of Columbia", "Delaware", "Florida", "Georgia", "Guam", 
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", 
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", 
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", 
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", 
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", 
"Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]
state_names = [x.lower() for x in state_names]

state_names.append("sentiment_num")
state_names.append("rawFeatures")

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = VectorAssembler(inputCols= state_names, outputCol="features")

lr = LinearRegression(featuresCol="features", labelCol="score")

pipeline_linear = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 13, Finished, Available, Finished)

In [10]:
(trainingData, testData) = df.randomSplit([0.7, 0.3])

model_linear = pipeline_linear.fit(trainingData)
predictions_linear = model_linear.transform(testData)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 15, Finished, Available, Finished)

In [13]:
datastore = 'azureml://datastores/workspaceblobstore/paths'
model_linear.save(f"{datastore}/models/ml_model_linear_score_prediction")

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 18, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o6977.save.
: java.io.IOException: Path azureml://datastores/workspaceblobstore/paths/models/ml_model_linear_score_prediction already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(events.scala:174)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent$(events.scala:169)
	at org.apache.spark.ml.util.Instrumentation.withSaveInstanceEvent(Instrumentation.scala:42)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$3(Pipeline.scala:344)
	at org.apache.sp

In [12]:
from pyspark.ml import PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator

PipelineModel.load(f"{datastore}/models/ml_model_score_prediction") 
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_linear)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 17, Finished, Available, Finished)

Root Mean Squared Error (RMSE) on test data = 68.5895


In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import PipelineModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=1000)
idf = VectorAssembler(inputCols= state_names, outputCol="features")

rf = RandomForestRegressor(featuresCol="indexedFeatures", labelCol = "score")

pipeline_forest = Pipeline(stages=[tokenizer, hashingTF, idf, rf])

(trainingData, testData) = df.randomSplit([0.7, 0.3])

model_forest = pipeline_forest.fit(trainingData)
predictions_forest = model_forest.transform(testData)

#PipelineModel.load(f"{datastore}/models/ml_model_score_prediction") 
evaluator = RegressionEvaluator(labelCol="score", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_forest)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

StatementMeta(09c8ba2d-49fa-498b-9d76-d59b947bd032, 55, 10, Submitted, Running, Running)

IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: {
  "name":%20%22workspaceblobstore%22,%0A%20%20%22container_name%22:%20%22azureml-blobstore-ab2acca4-77bf-4607-9fc6-941bb7fd6768%22,%0A%20%20%22account_name%22:%20%22projectgstorageef8131633%22,%0A%20%20%22protocol%22:%20%22https%22,%0A%20%20%22endpoint%22:%20%22core.windows.net%22%0A%7D/models/ml_model_score_prediction/metadata